In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv("/kaggle/input/us-counties-covid-19-dataset/us-counties.csv")

data['date'] = pd.to_datetime(data['date'])
data['month']= data.date.dt.to_period('M')
data2= data[data.county!='Unknown']
print(data2.head())

data2['month_name'] = data2['month'].dt.strftime('%b')
print(data2.head())

Import and inspect the latest COVID-19 case data file by county, date and state.

In [ ]:
data3=data2.groupby('date').cases.sum().reset_index()

fig,ax = plt.subplots(figsize=(8, 4)) 
sns.set_style("whitegrid")
sns.set_context("paper")
sns.lineplot(data=data3, x="date", y="cases")
plt.title("Number of Cumulative COVID19 Cases in United States")
plt.xlabel("Month")
plt.ylabel("Number of Cases")
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000000) + 'M'))

plt.show()

**As of August 2020, the number of COVID19 cases in U.S. has surpassed 4M and is approaching 5M. Out of all months, July saw the largest growth in new cases.** 

In [ ]:
data4=data2.groupby('date').deaths.sum().reset_index()

fig,ax = plt.subplots(figsize=(8, 4)) 
sns.set_style("whitegrid")
sns.lineplot(data4.date,data4.deaths, color='red')
plt.title("Number of Cumulative COVID19 Deaths in United States")
plt.xlabel("Month")
plt.ylabel("Number of Deaths")
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))

plt.show()

**There have been around 160k COVID related deaths in U.S. April was the 'deadliest' month, largely driven by the outbreak in NYC. May and August placed #2 and #3.**

In [ ]:
total_cases=data2[data2.date=='2020-07-31']
cases_by_state=total_cases.groupby('state').cases.sum().reset_index()
cases_by_state=cases_by_state.sort_values('cases', ascending=False).reset_index()
print(cases_by_state.head())

fig,ax = plt.subplots(figsize=(20,7))
sns.set_context("notebook")
sns.barplot(data=cases_by_state, x=cases_by_state.state, y=cases_by_state.cases)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))
plt.xticks(rotation=90)
plt.title("Number of Cumulative COVID19 Cases by State")

plt.show()

**California, Florida, Texas,and NY are the top 4 states by a landslide, with the most cumulative COVID cases. **

In [ ]:
death_by_state=total_cases.groupby('state').deaths.sum().reset_index()
death_by_state=death_by_state.sort_values('deaths', ascending=False).reset_index()
print(death_by_state.head())

fig,ax = plt.subplots(figsize=(20,7))
sns.set_context("notebook")
sns.barplot(data=death_by_state, x='state', y='deaths')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))
plt.xticks(rotation=90)
plt.title("Number of Cumulative COVID19 Deaths by State")

plt.show()

**However, NY still accounts for the most COVID deaths, more than doubling vs. any other states. **

In [ ]:
casesv2=data2.groupby(['state','county','month_name']).cases.max().reset_index()
cases_latest=casesv2[(casesv2.month_name== 'Jun') | (casesv2.month_name== 'Jul')].reset_index()
cases_latest['countyv2']=cases_latest[['state', 'county']].agg('-'.join, axis=1)
print(cases_latest.head())

pivoted2=pd.pivot_table(cases_latest, values='cases', index=['countyv2'], columns=['month_name'], fill_value=0, aggfunc=np.sum).reset_index()
print(pivoted2.head())

pivoted2['new_cases_in_July']=pivoted2['Jul']-pivoted2['Jun']
pivoted2['growth%']=100*pivoted2['new_cases_in_July']/pivoted2['Jun']
pivoted2=pivoted2.sort_values('new_cases_in_July', ascending=False).reset_index()
print(pivoted2.head())

pivoted2['growth%']=pivoted2['growth%'].values.astype(np.int64)
print(pivoted2['growth%'].dtypes)

pivoted3=pivoted2.sort_values('growth%', ascending=False).reset_index()
print(pivoted3.head())

pivoted_top20_cases=pivoted2.loc[0:19]
print(pivoted_top20_cases)
pivoted_top20_rates=pivoted3.loc[0:19]
print(pivoted_top20_rates)

In [ ]:
fig,ax = plt.subplots(figsize=(12,5))
sns.set_context("notebook")
sns.barplot(data=pivoted_top20_cases, x='countyv2', y='new_cases_in_July')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))
plt.xticks(rotation=90)
plt.title("Top 20 U.S. Counties by Number of New COVID Cases in July")
plt.xlabel('State-County')
plt.ylabel('Number of New Cases in July')
plt.show()

fig,ax = plt.subplots(figsize=(12,5))
sns.set_context("notebook")
sns.barplot(data=pivoted_top20_rates, x='countyv2', y='growth%')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K%'))
plt.xticks(rotation=90)
plt.title("Top 20 U.S. Counties by New Case Growth Rate in July")
plt.xlabel('State-County')
plt.ylabel('Growth Rate of New Cases in July')
plt.show()

**Examining the latest data in July by county, Los Angeles topped all other counties in terms of number of new cases. Among the top 20 counties with the most new cases, California and Florida each accounted for 6, followed by Texas with 5. However, the top 20 fastest-growing COVID hotspots tell a different story. The growth rates are exponential- over 1000%! Also most of them are less populated areas. 7 out of the top 20 are in Texas.** 

In [ ]:
california=data2[data2.state=='California']
cases_by_county=california.groupby(['county','month']).cases.max().reset_index()
print(cases_by_county.head())

fig,ax = plt.subplots(figsize=(25,7))
sns.barplot(data=cases_by_county, x='county', y='cases', hue='month')
plt.xticks(rotation=90)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))
plt.title("Number of Cumutive COVID19 Cases by County- California")
plt.legend(loc=1)

plt.show()

**In California, Los Angeles county stand out the most in number of COVID cases (nearly 200k as of August).** 

In [ ]:
la=data2[data2.county=="Los Angeles"]
print(la.head())

fig,ax = plt.subplots(figsize=(8, 4))
plt.plot(la.date, la.cases,color='blue')
plt.plot(la.date, la.deaths, color='red')
plt.title('COVID19 Cases and Deaths in Los Angeles County')
plt.xlabel('date')
plt.ylabel('number')
plt.legend(['Cases','Deaths'])
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))

plt.show()

**Los Angeles saw the most increase in new cases in July. Half of the cumulative cases occured in July (100K). This is likely due to the wider availability of testing and reopening of higher-risk businesses in June (restaurants, gyms, etc). However, the silver lining is the death rate is remaining low. **

In [ ]:
la2= la.groupby('month_name').agg({'cases': 'max', 'deaths': 'max'}).reset_index()
print(la2)

la3=la2.set_index('month_name').transpose()

la3['new_in_march']= (la3['Mar']-la3['Feb'])
la3['new_in_april']= la3['Apr']-la3['Mar']
la3['new_in_may']= la3['May']-la3['Apr']
la3['new_in_june']= la3['Jun']-la3['May']
la3['new_in_july']= la3['Jul']-la3['Jun']
la3['new_in_august']= la3['Aug']-la3['Jul']
la3=la3.reset_index(drop=True)
la3.loc['death rate']=100*la3.iloc[1]/la3.iloc[0]
print(la3)

la_death_rate=la3.iloc[2]
la_death_rate2=la_death_rate[['new_in_march','new_in_april','new_in_may','new_in_june','new_in_july','new_in_august']]
print(la_death_rate2)

month_list=['March', "April","May","June","July","August"]

fig,ax = plt.subplots(figsize=(8, 4))
plt.plot(la_death_rate2, linestyle="--", marker="o", color="red")
plt.title('COVID19 Death Rate in Los Angeles County')
plt.xlabel('Month')
plt.ylabel('Death Rate')
ax.set_xticklabels(month_list)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.1f}'.format(x) + '%'))

plt.show()

**The COVID19 death rate peaked in April, mostly likely due to a combination of factors including less hospitalization/medical resources, limited testing and higher susceptibility to the disease among older demographic at the beginning of the outbreak. **